In [1]:
%load_ext cudf.pandas
import pandas as pd

print(pd)

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>


In [2]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import glob
from sklearn.model_selection import train_test_split

In [3]:
import torch
torch.__version__

'2.1.0.post300'

In [4]:
articles = pd.read_csv("data/articles.csv")
customers = pd.read_csv("data/customers.csv")
transactions = pd.read_csv("data/transactions_train.csv")

In [5]:
print("articles shape: ", articles.shape)
print("customers shape: ", customers.shape)
print("transactions shape: ", transactions.shape)

articles shape:  (105542, 25)
customers shape:  (1371980, 7)
transactions shape:  (31788324, 5)


In [6]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [7]:
transactions['customer_id'] = transactions['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
transactions['article_id'] = transactions.article_id.astype('int32')
#train['article_id'] = '0' + train.article_id.astype('str')
transactions.t_dat = pd.to_datetime(transactions.t_dat)
transactions = transactions[['t_dat','customer_id','article_id']]
transactions['year'] = (transactions.t_dat.dt.year-2000).astype('int8')
transactions['month'] = (transactions.t_dat.dt.month).astype('int8')
transactions['day'] = (transactions.t_dat.dt.day).astype('int8')
del transactions['t_dat']
# transactions.to_parquet('transactions.pqt',index=False)
print( transactions.shape )
transactions.head()

(31788324, 5)


,customer_id,article_id,year,month,day
0,-6846340800584936,663713001,18,9,20
1,-6846340800584936,541518023,18,9,20
2,-8334631767138808638,505221004,18,9,20
3,-8334631767138808638,685687003,18,9,20
4,-8334631767138808638,685687004,18,9,20


In [8]:
customers['customer_id'] = customers['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
articles['article_id'] = articles['article_id'].astype('int32')

In [9]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,6883939031699146327,NaN,NaN,ACTIVE,NONE,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,-7200416642310594310,NaN,NaN,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,-6846340800584936,NaN,NaN,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,-94071612138601410,NaN,NaN,ACTIVE,NONE,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,-283965518499174310,1.0,1.0,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [10]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [11]:
transactions.article_id.nunique()

104547

In [12]:
transactions.customer_id.nunique()

1362281

In [13]:
len(customers)

1371980

In [14]:
len(customers)-transactions.customer_id.nunique()

9699

In [15]:
customers.customer_id.nunique()

1371980

In [16]:
transactions.groupby(['year', 'month']).count()

customer_id  article_id      day
year month                                  
18   9           594776      594776   594776
     10         1397040     1397040  1397040
     11         1270619     1270619  1270619
     12         1148827     1148827  1148827
19   1          1263471     1263471  1263471
     2          1152412     1152412  1152412
     3          1286750     1286750  1286750
     4          1476454     1476454  1476454
     5          1560319     1560319  1560319
     6          1906202     1906202  1906202
     7          1807494     1807494  1807494
     8          1253530     1253530  1253530
     9          1227178     1227178  1227178
     10         1146772     1146772  1146772
     11         1198033     1198033  1198033
     12         1118315     1118315  1118315
20   1          1076354     1076354  1076354
     2          1001859     1001859  1001859
     3          1047752     1047752  1047752
     4          1340882     1340882  1340882
     5          1361815     1361815  1361815
     6          1764507     1764507  1764507
     7          1351502     1351502  1351502
     8          1237192     1237192  1237192
     9           798269      798269   798269

In [24]:
transactions = transactions[(transactions['year']>=20) & (transactions['month']>=8)]
transactions.shape

(2035461, 5)

In [25]:
customers = customers[customers['customer_id'].isin(set(transactions.customer_id.unique()))]
print(len(customers))

363798


In [26]:
articles = articles[articles['article_id'].isin(set(transactions.article_id.unique()))]
print(len(articles))

34870


In [27]:
articles.reset_index(inplace=True)
customers.reset_index(inplace=True)

In [28]:
all_image_paths = glob.glob(f"data/images/*/*")
print(len(all_image_paths))
all_image_ids = set()
for path in tqdm(all_image_paths):
    article_id = path.split('/')[-1].split('.')[0]
    all_image_ids.add(int(article_id))

105100


  0%|          | 0/105100 [00:00<?, ?it/s]

In [29]:
images_path = f'data/images/0'

articles["path"] = images_path + articles["product_code"].astype(str) + "/0" + articles["article_id"].astype(str) + ".jpg"

# Adjust the incorrect paths and set them to None
for k, article_id in tqdm(enumerate(articles["article_id"])):
    if article_id not in all_image_ids:
        articles.loc[k, "path"] = None

0it [00:00, ?it/s]

In [30]:
articles.dropna(subset=['path'],inplace=True)

In [31]:
articles.shape

(34814, 27)

In [32]:
transactions

,customer_id,article_id,year,month,day
29752863,1130996224714012477,743123001,20,8,1
29752864,1130996224714012477,743123001,20,8,1
29752865,1130996224714012477,849597003,20,8,1
29752866,1130996224714012477,599580052,20,8,1
29752867,1130996224714012477,599580052,20,8,1
...,...,...,...,...,...
31788319,4685485978980270934,929511001,20,9,22
31788320,4685485978980270934,891322004,20,9,22
31788321,3959348689921271969,918325001,20,9,22
31788322,-8639340045377511665,833459002,20,9,22


In [33]:
core_10_customers = transactions.groupby(['customer_id'])['article_id'].count()>=10
core_10_customers

customer_id
-9223352921020755230    False
-9223343869995384291     True
-9223290575350349271    False
-9223279922255655589    False
-9223251502792607675    False
                        ...  
 9222733909628763172    False
 9222940818992675193    False
 9223099843213569889    False
 9223141695752178477    False
 9223144259947635662    False
Name: article_id, Length: 363798, dtype: bool

In [34]:
core_10_customers[core_10_customers].index

Int64Index([-9223343869995384291, -9223100958908512198, -9222973878991087276,
            -9222798684148120334, -9222465182417539881, -9221950529560072627,
            -9221661208958206525, -9221250204668709066, -9220984377508749635,
            -9220328629218950774,
            ...
             9219838591013150561,  9219928155584075153,  9220164111821453273,
             9220246776645318990,  9221013759061785384,  9221380166206241768,
             9221796771717283757,  9222484380911032362,  9222546307854699030,
             9222665711703637549],
           dtype='int64', name='customer_id', length=55854)

In [35]:
transactions = transactions[transactions['customer_id'].isin(set(core_10_customers[core_10_customers].index))]

In [36]:
core_10_articles = transactions.groupby(['article_id'])['customer_id'].count()>=10
core_10_articles

article_id
108775044     True
110065002    False
110065011    False
111565001     True
111586001     True
             ...  
952267001     True
952938001    False
953450001    False
953763001     True
956217002     True
Name: customer_id, Length: 29792, dtype: bool

In [37]:
core_10_articles[core_10_articles].index

Int64Index([108775044, 111565001, 111586001, 111593001, 111609001, 123173001,
            129085001, 129085026, 129085027, 130035001,
            ...
            947168001, 947509001, 947599001, 947934001, 949198001, 949551001,
            949551002, 952267001, 953763001, 956217002],
           dtype='int64', name='article_id', length=12841)

In [38]:
transactions = transactions[transactions['article_id'].isin(set(core_10_articles[core_10_articles].index))]

In [39]:
merged1 = pd.merge(transactions, articles, on='article_id', how='inner')
merged2= pd.merge(transactions, customers, on='customer_id', how='inner')
common_articles = set(merged1['article_id'])
common_customers = set(merged2['customer_id'])
transactions = transactions[transactions['article_id'].isin(common_articles) & transactions['customer_id'].isin(common_customers)]
common_articles = set(transactions['article_id'])
common_customers = set(transactions['customer_id'])
articles = articles[articles['article_id'].isin(common_articles)]
customers = customers[customers['customer_id'].isin(common_customers)]

In [40]:
print(transactions.article_id.nunique())
print(articles.article_id.nunique())
print(transactions.customer_id.nunique())
print(customers.customer_id.nunique())

12823
12823
55845
55845


In [41]:
transactions.shape

(883196, 5)

In [42]:
customer_map = {u: i for i, u in enumerate(set(transactions['customer_id']))}
article_map = {a: i for i, a in enumerate(set(transactions['article_id']))}
transactions['customer_id'] = transactions['customer_id'].map(customer_map)
transactions['article_id'] = transactions['article_id'].map(article_map)
customers['customer_id'] = customers['customer_id'].map(customer_map)
articles['article_id'] = articles['article_id'].map(article_map)


In [43]:
transactions.shape

(883196, 5)

In [44]:
transactions.customer_id.nunique()

55845

In [45]:
customers.shape

(55845, 8)

In [46]:
articles.shape

(12823, 27)

In [47]:
transactions.reset_index(inplace=True, drop=True)
customers.reset_index(inplace=True)
articles.reset_index(inplace=True)

In [48]:
transactions.head()

,customer_id,article_id,year,month,day
0,26873,3946,20,8,1
1,26873,3946,20,8,1
2,26873,8232,20,8,1
3,26873,9360,20,8,1
4,26873,9360,20,8,1


In [49]:
train, test = train_test_split(transactions, test_size=0.2, random_state=42, stratify=transactions['article_id'])

In [50]:
train.shape, test.shape

((706556, 5), (176640, 5))

In [51]:
test

,customer_id,article_id,year,month,day
613137,31399,4726,20,9,5
189672,26028,8887,20,8,11
77693,19440,6773,20,8,4
799883,1011,3503,20,9,17
86335,10819,6045,20,8,5
...,...,...,...,...,...
492884,32952,635,20,8,29
633383,42217,685,20,9,6
715096,23164,6240,20,9,11
749303,45695,2133,20,9,13


In [52]:
transactions.head()

,customer_id,article_id,year,month,day
0,26873,3946,20,8,1
1,26873,3946,20,8,1
2,26873,8232,20,8,1
3,26873,9360,20,8,1
4,26873,9360,20,8,1


In [53]:
transactions.to_csv("data/transactions.csv", index=False)
train.to_csv("data/train.csv")
test.to_csv("data/test.csv")

In [54]:
def transaction_only(df):
    df = df.sort_values(by=['customer_id','article_id'])
    df['article_id'] = df['article_id'].map(str)
    df = df.groupby(['customer_id']).agg({'article_id':' '.join})
    return df
train = transaction_only(train)
test = transaction_only(test)
train
                        

,article_id
customer_id,
0,341 471 1170 1282 2183 2768 3673 3852 4476 514...
1,430 608 1129 2045 4177 5120 5430 6253 6989 845...
2,707 1480 1582 3472 3571 3571 4342 6208 6392 63...
3,1283 2141 3285 4345 4935 4938 5029 5061 5771 5...
4,3079 5070 8598 9192 11445
...,...
55840,259 259 260 260 1280 2041 2275 2275 2511 3394 ...
55841,850 1041 1226 2323 2511 2929 4634 4634 4635 51...
55842,22 241 1062 1340 1585 2721 4065 5432 6684 8098...


In [55]:
f = open('data/train.txt', 'w')
for customer, articles in enumerate(train.values):
    f.write(f'{customer} {articles[0]}\n')

In [56]:
f = open('data/test.txt', 'w')
for customer, articles in enumerate(test.values):
    f.write(f'{customer} {articles[0]}\n')